In [40]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate

import cv2
import time
import math
import sys
import csv

sys.path.append("./protos")

import grpc
import messaging_pb2
import messaging_pb2_grpc

class ObjectDetectionLayer:
    def __init__(
        self, weights_file=None, classes_file=None, config_file=None, min_confidence=0.3
    ):
        self.weights_file = weights_file
        self.config_file = config_file
        self.classes_file = classes_file

        self.net = self._load_model()
        self.min_confidence = min_confidence

        self.classes = ["car", "truck", "bus", "minibus", "cyclist"]

    def _load_model(self):
        return cv2.dnn.readNet(self.weights_file, self.config_file)

    def _get_output_layers(self, net):
        layer_names = net.getLayerNames()
        try:
            output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
        except:
            output_layers = [
                layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()
            ]

        return output_layers

    def _get_bboxes_pixels(self, img):
        image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        Height, Width = image.shape[:2]
        scale = 0.00392

        blob = cv2.dnn.blobFromImage(
            image, scale, (416, 416), (0, 0, 0), True, crop=False
        )

        self.net.setInput(blob)

        outs = self.net.forward(self._get_output_layers(self.net))

        class_ids = []
        confidences = []
        boxes = []
        conf_threshold = 0.5
        nms_threshold = 0.4

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > self.min_confidence:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

        bboxes_with_confidence = []
        for i in indices:
            try:
                box = boxes[i]
            except Exception:
                box = boxes[i[0]]

            x, y, w, h = [max(v, 0) for v in box[:4]]  # model outputs can be negative

            bboxes_with_confidence.append(
                np.array((x, x + w, y, y + h, 100 * confidences[i]))
            )

        # follows the format of x0, x1, y0, y1, confidence
        return np.array(bboxes_with_confidence).astype(int)

    def _bbox_pixels_to_gps(self, bboxes, gps_corners, img_dim):
        top_left_gps, top_right_gps, bot_left_gps, bot_right_gps = gps_corners
        right_vec = (top_right_gps - top_left_gps) / img_dim[1]
        bot_vec = (bot_left_gps - top_left_gps) / img_dim[0]

        transformation = np.array(
            [[right_vec[0], bot_vec[0]], [right_vec[1], bot_vec[1]]]
        )

        def transform(bbox):
            return transformation @ bbox + top_left_gps

        return np.array([(*transform(bbox[:2]), *bbox[2:4]) for bbox in bboxes])

    # Be careful using this radius. Pixel distance does not convert to latitude and longitude in the same way
    def _convert_bbox_to_radial_representation(self, bboxes):
        """
        Given a bbox in the format of x0, x1, y0, y1, confidence.
        Returns a bbox in the format of x, y, r, confidence
        """

        # TODO: Scale by some factor (function of altitude) to convert pixels to meters
        dx = bboxes[:, 1] - bboxes[:, 0]
        dy = bboxes[:, 3] - bboxes[:, 2]
        radii = np.sqrt(dx**2 + dy**2) / 2

        return np.array(
            [
                (bboxes[:, 0] + bboxes[:, 1]) / 2,
                (bboxes[:, 2] + bboxes[:, 3]) / 2,
                radii,
                bboxes[:, 4],
            ]
        ).T

    def run(self, img, gps_sample_corners):
        bboxes_pixels = self._get_bboxes_pixels(img)

        if len(bboxes_pixels) == 0:
            return [], []

        bboxes_radial_pixels = self._convert_bbox_to_radial_representation(
            bboxes_pixels
        )
        bboxes_radial_gps = self._bbox_pixels_to_gps(
            bboxes_radial_pixels, gps_sample_corners, img.shape
        )

        return bboxes_radial_gps, bboxes_pixels  # remove the pixels bboxes later


class MavlinkInterfaceLayer:
    def __init__(self, protos_path="protos"):
        self.protos_path = protos_path
        self.channel = grpc.insecure_channel("localhost:50051")
        self.stub = messaging_pb2_grpc.MessagingServiceStub(self.channel)

    def run(self, bboxes):
        if len(bboxes) == 0:
            return

        responses = []

        for bbox in bboxes:
            encoded_bbox = str(bbox)[1:-1] # remove the brackets
            response = self.stub.SendData(messaging_pb2.DataRequest(data=encoded_bbox))
            responses.append(response)

        return responses

In [41]:
import geopy
import geopy.distance

def _destination_point(start_lat, start_lon, bearing, distance):
        start_point = geopy.Point(start_lat, start_lon)
        distance = geopy.distance.distance(meters=distance)
        destination_point = distance.destination(point=start_point, bearing=bearing)
        return destination_point.latitude, destination_point.longitude

In [61]:
import os
filenames = sorted(os.listdir("../data_ignore/sequential_data"))

In [64]:
import exiftool

img_layer_replacement = []
directory_path = "../data_ignore/sequential_data/"


for image_name in filenames[197:210]:
    
    image_path = directory_path + image_name
    print(f"calculating GPS corners for image: {image_path}")
    image = np.asarray(Image.open(image_path))[:, :, :3]
    with exiftool.ExifToolHelper() as et:
        metadata = et.get_metadata(image_path)[0]

    yaw = metadata["XMP:GimbalYawDegree"]
    if isinstance(yaw, str):
        yaw = float(yaw[1:])

    latitude = metadata["EXIF:GPSLatitude"]
    longitude = metadata["EXIF:GPSLongitude"]

    altitude = metadata["XMP:RelativeAltitude"] # meters
    if isinstance(altitude, str):
        altitude = float(altitude[1:])

    radians = math.radians(69.4)
    base = 2*altitude*math.tan(radians/2)
    width = base
    height = base * 3/4

    sequential_image_height = 3000
    sequential_image_width = 4000

    meters_per_pixel = width / sequential_image_width

    si_top_gps = _destination_point(latitude, longitude, yaw, height/2)
    si_top_right_gps = _destination_point(si_top_gps[0], si_top_gps[1], yaw+90, width/2)
    si_top_left_gps = _destination_point(si_top_gps[0], si_top_gps[1], yaw-90, width/2)

    si_bot_gps = _destination_point(latitude, longitude, yaw+180, height/2)
    si_bot_right_gps = _destination_point(si_bot_gps[0], si_bot_gps[1], yaw+90, width/2)
    si_bot_left_gps = _destination_point(si_bot_gps[0], si_bot_gps[1], yaw-90, width/2)

    image_corner_gps = (si_top_left_gps, si_top_right_gps, si_bot_left_gps, si_bot_right_gps)

    img_layer_replacement.append((image, image_corner_gps))

calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0199.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0200.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0201.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0202.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0203.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0204.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0205.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0206.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0207.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0208.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0209.JPG
calculating GPS corners for image: ../data_ignore/sequential_data/DJI_0210.JPG
calculating GPS corners for image: ../data_ignore/se

In [73]:
weights_file = "../yolo/yolov3-aerial.weights"
classes_file = "../yolo/aerial.names"
config_file = "../yolo/yolov3-aerial.cfg"

obj_layer = ObjectDetectionLayer(
    config_file=config_file, weights_file=weights_file, classes_file=classes_file
)

mav_layer = MavlinkInterfaceLayer()

for img, img_corner_gps in img_layer_replacement:
    img_corner_gps = np.array(img_corner_gps)
    bboxes_gps, bboxes_pixels = obj_layer.run(img, img_corner_gps)

    for bbox_gps in bboxes_gps:
        bbox_gps[2] = bbox_gps[2] * meters_per_pixel

    responses = mav_layer.run(bboxes_gps)
    
    if len(bboxes_gps) != 0 and len(responses) == 0:
        print("No responses from MAVLink")

    if responses: 
        print(bboxes_pixels)

[[882 936  15  84  61]]
[[ 176  310  706  877   99]
 [2297 2395   15   71   63]]
[[ 197  303 1136 1328   96]
 [ 616  748  255  414   86]
 [ 638  700   61  157   77]
 [ 334  506  823 1097   75]]
[[ 807  908  307  457   98]
 [ 981 1090  109  227   62]]
[[ 877  996 2652 2850   96]
 [1321 1460   25  179   88]
 [1514 1585    0   88   52]]
[[1045 1166 2414 2637   96]]
[[2183 2308  104  306   99]
 [1906 2021  754  932   98]
 [1752 1846 1171 1350   94]
 [1843 1971 1218 1389   82]
 [2028 2116 2600 2677   70]]
[[1652 1771 2352 2547   98]
 [2117 2222 1310 1467   89]
 [1546 1616 2322 2414   76]]
[[2701 2751  388  496   90]]
[[2537 2649 1888 2058   96]
 [2926 2993  538  629   94]
 [3728 3857  783  974   73]]
[[2624 2739 2216 2377   95]
 [3275 3386 1035 1249   90]
 [2466 2591 2688 2862   78]
 [3255 3331  557  655   52]]
